In [1]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
from pudl import ferc1
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
def smallwind(wind_plants,f1_engine):
    small_plant = pd.read_sql('''SELECT p.respondent_id, r.respondent_name, p.report_year, p.plant_name,\
                                p.yr_constructed, p.capacity_rating, p.net_demand, p.net_generation, p.plant_cost,\
                                p.plant_cost_mw, p.operation, p.expns_fuel, p.expns_maint, p.kind_of_fuel,\
                                p.fuel_cost\
                                FROM f1_gnrt_plant p JOIN f1_respondent_id r ON r.respondent_id=p.respondent_id''',f1_engine)

    wind_name = small_plant[small_plant['plant_name'].str.contains("wind", case=False)]
    wind_fuel = small_plant[small_plant['kind_of_fuel'].str.contains("wind", case=False)]
    wind_plants = pd.concat([wind_plants,wind_name,wind_fuel])
    wind_plants.drop_duplicates(inplace=True)
    wind_plants = wind_plants[(wind_plants.yr_constructed!='')]
    return(wind_plants)

In [ ]:
wind_plants = pd.DataFrame()
for year in range(2004,2016):
    print('Initializing FERC Form 1 DB for {}'.format(year))
    ferc1.init_db(refyear=2015,years=[year,])
    f1_engine = create_engine('postgresql://catalyst@localhost:5432/ferc1')
    wind_plants = smallwind(wind_plants, f1_engine)

wind_plants.to_csv('ferc1_wind_plants_2004-2015.csv')